In [68]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# load MATLAB files
from scipy.io import loadmat

In [61]:
data = loadmat('data/ex4data1.mat')
X = data['X'];
ones = np.ones((X.shape[0],1))
X = np.c_[ones,X]
y = data['y']
print("Shape of X (with intercept) :",X.shape)
print("Shape of y ",y.shape)
weights = loadmat('data/ex4weights.mat')
theta1 = weights['Theta1']
theta2 = weights['Theta2']
print("\n")
print("Shape of Theta 1:",theta1.shape)
print("Shape of Theta 2:",theta2.shape)

Shape of X (with intercept) : (5000, 401)
Shape of y  (5000, 1)


Shape of Theta 1: (25, 401)
Shape of Theta 2: (10, 26)


In [62]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [6]:
def sigmoid_gradient(z):
    return sigmoid(z)*(1-sigmoid(z))

In [17]:
def randInitializeWeights(L_in, L_out):
    epsilion_init = 0.12
    W = np.random.randn(L_out,L_in+1) * 2 *epsilion_init - epsilion_init
    return W

In [43]:
# sigmoid(0)
# print(randInitializeWeights(2,2))

In [48]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda):
    Theta1 = nn_params[0:(input_layer_size+1)*(hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
    Theta2 = nn_params[(input_layer_size+1)*(hidden_layer_size):].reshape(num_labels,hidden_layer_size+1)
    
    y_matrix = pd.get_dummies(y.ravel()).as_matrix()
    
#     forward propogation
    z2 = np.dot(X,Theta1.T)
    a2 = np.c_[np.ones((z2.shape[0],1)),sigmoid(z2)]
    z3 = np.dot(a2,Theta2.T)
    a3 = sigmoid(z3)
     
    m = a3.shape[0]
    
    J = -1*(1/m)*np.sum((y_matrix*np.log(a3))+((1-y_matrix)*np.log(1-a3)))+(Lambda/(2*m))*(np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])))
#     print(J)
  
#     print(a3.shape)
#     print(Theta2.shape)
#     print(z2.shape)
    delta3 = a3 - y_matrix
    delta2 = np.multiply(np.dot(delta3,Theta2[:,1:]),sigmoid_gradient(z2))
    
    D2 = np.dot(delta3.T,a2)
    D1 = np.dot(delta2.T,X)
    
    theta1_reg=np.c_[np.ones((Theta1.shape[0],1)),Theta1[:,1:]]
    theta2_reg=np.c_[np.ones((Theta2.shape[0],1)),Theta2[:,1:]]
    
    theta1_grad = (1/m)*D1+(1/m)*(theta1_reg*Lambda)
    theta2_grad = (1/m)*D2+(1/m)*(theta2_reg*Lambda)
    
    grad = np.r_[theta1_grad.ravel(),theta2_grad.ravel()]
    
    return (J,grad)
    

In [54]:
input_layer_size = 400
hidden_layer_size = 25
num_labels = 10
Lambda = 0
nn_params = np.hstack((theta1.ravel(),theta2.ravel()))
#print(nn_params.shape)
nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda)
Lambda = 1
nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda)

(0.38376985909092365,
 array([  2.61871277e-04,  -2.11248326e-12,   4.38829369e-13, ...,
          4.70513145e-05,  -5.01718610e-04,   5.07825789e-04]))

In [92]:
def compute_numerical_gradient(theta, lmda):
    eps = 10**(-4)
    thetaSize = theta.shape[0]
    gradTheta = np.zeros(theta.shape)
    epsTheta = np.zeros(theta.shape)
    for i in range(0,thetaSize):
        epsTheta[i] = eps
        J_plus = nnCostFunction(theta + epsTheta, input_layer_size, hidden_layer_size, num_labels, X, y, lmda)[0]
        J_minus = nnCostFunction(theta - epsTheta, theta1.shape[1]-1, theta1.shape[0], theta2.shape[0], X, y, lmda)[0]
        gradTheta[i] = (J_plus - J_minus)/(2*eps)
        epsTheta[i] = 0
    return gradTheta

In [93]:
grdient_nn = compute_numerical_gradient(nn_params, 0)
print(grdient_nn)

[  6.18712781e-05   0.00000000e+00   0.00000000e+00 ...,   9.66104735e-05
  -7.57736845e-04   7.73329873e-04]


In [94]:
gradient = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 0)[1]

[  6.18712766e-05   0.00000000e+00   0.00000000e+00 ...,   9.66104721e-05
  -7.57736846e-04   7.73329872e-04]


In [96]:
#print(nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 0)[1])
# print(gradient.shape)
# print(grdient_nn.shape)
# for i in range(0,gradient.shape[0]):
#     print(gradient[i],grdient_nn[i])
#     print("\n")


In [98]:
lmda = 1
input_layer_size = 400
hidden_layer_size = 25
num_labels = 10
theta_1 = randInitializeWeights(input_layer_size,hidden_layer_size)
theta_2 = randInitializeWeights(hidden_layer_size,num_labels)
theta_0 = np.hstack((theta1.ravel(),theta2.ravel()))
nn = minimize( fun = nnCostFunction, x0 = theta_0, 
              args = (input_layer_size, hidden_layer_size, num_labels, X, y, lmda), 
              method = 'CG', jac = True, options = {'maxiter' : 400} )
print(nn)

     fun: 0.2234818267273173
     jac: array([  2.87468705e-05,  -7.37022186e-14,   1.53102742e-14, ...,
        -6.86126185e-05,  -4.84402082e-05,   2.74205845e-06])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 430
     nit: 152
    njev: 419
  status: 2
 success: False
       x: array([ -1.16095040e+00,  -3.68511093e-10,   7.65513712e-11, ...,
         9.47867525e-02,   2.39551341e+00,  -1.68812815e+00])


In [118]:
def predict(theta1,theta2,X):
    z2 = np.dot(X,theta1.T)
    a2 = sigmoid(z2)
    a2 = np.c_[np.ones((a2.shape[0],1)),a2]
    z3 = np.dot(a2,theta2.T)
    a3 = sigmoid(z3)
    return (np.argmax(a3,axis=1)+1)

In [128]:
theta_0 = nn.x[0:(hidden_layer_size*(input_layer_size + 1))].reshape(hidden_layer_size,(input_layer_size+1))
theta_1 = nn.x[(hidden_layer_size*(input_layer_size + 1)):].reshape(num_labels,(hidden_layer_size + 1))
predict_y = predict(theta_0,theta_1,X).reshape(-1,1)
probability = (y[y == predict_y].shape[0]/(y.shape[0]))*100
print("training accuracy : "+str(probability)+" %")

training accuracy : 99.44 %
